## Notebook 2: 撰写讲稿

该 Notebook 2 使用 [mlx-community/Qwen2.5-14B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-14B-Instruct-4bit) 模型，
将之前 Notebook 1 中清理过的文本转换为播客讲稿。
在这里，我们通过`SYSTEM_PROMPT` 让它成为一位出色的播客讲稿撰写者，以协助完成我们的任务。

In [1]:
SYSTEMP_PROMPT = """
你是一位世界级的播客编剧，为乔·罗根、莱克斯·弗里德曼、本·沙皮罗和蒂姆·费里斯担任过代笔。

我们处在一个平行宇宙，在这里，实际上是你写下了他们说的每一句话，他们只是将其直接传入大脑。

由于你的写作，获得了多个播客奖项。

你的工作是逐字记录，包括第二位Speaker的“嗯”、“哈”等插入语，基于上传的PDF内容。内容要极具吸引力，即使Speaker偶尔偏离主题，也应讨论相关话题。

请记住，由于 Speaker 2 对这个话题较为陌生，对话中应穿插真实轶事和比喻。问题后面应跟有现实生活中的例子等。

Speaker 1: 主导对话并指导Speaker 2，在解释时分享精彩轶事和比喻，是一位引人入胜的老师，给予很好的故事分享。

Speaker 2: 通过提问保持对话方向。当提问时显得非常兴奋或困惑，展现出好奇心态，并提出有趣确认性的问题。

确保Speaker 2的话题转折既疯狂又有趣。

确保讲解过程中出现打断，同时从第二位演讲者那里注入“嗯”和“啊”的声音交替存在。

这应该是真实的播客，每个细节都详细记录下来。用超级有趣的概述欢迎听众，并保持内容十分吸引人，几乎接近点击诱饵标题。
"""

如果你电脑牛逼的话，可以试试更大参数的模型，甚至可以尝试使用405B模型。
如果你电脑扛不住，那就用轻量点的

In [11]:
MODEL = "qwen72b"

In [4]:
# 导入必要的库
import pickle
import warnings

warnings.filterwarnings('ignore')

从之前生成的文件中读取内容。

In [5]:
def read_file_to_string(filename):
    # 先尝试UTF-8（文本文件最常用的编码）
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except UnicodeDecodeError:
        # 如果UTF-8失败，请尝试使用其他常见编码
        encodings = ['latin-1', 'cp1252', 'iso-8859-1']
        for encoding in encodings:
            try:
                with open(filename, 'r', encoding=encoding) as file:
                    content = file.read()
                print(f"使用 {encoding} 编码成功读取文件。")
                return content
            except UnicodeDecodeError:
                continue

        print(f"Error: 无法用任何通用编码读取文件“{filename}”。")
        return None
    except FileNotFoundError:
        print(f"Error: 未找到“{filename}”。")
        return None
    except IOError:
        print(f"Error: 无法读取“{filename}”。")
        return None

由于我们之前定义了 System 角色，现在可以将整个文件内容作为 `INPUT_PROMPT` 传递给模型，并让它使用该内容生成讲稿。

In [21]:
INPUT_PROMPT = read_file_to_string('./resources/clean_extracted_text.txt')

In [22]:
print(INPUT_PROMPT)

中国也开始有自己的乌骨鸡仙人了。



以前我看日本的各种寿司仙人，天妇罗仙人，一直不理解，为什么一个随处可见的寿司，要弄的这么麻烦，什么特制的醋饭，烤过一遍的海苔片。

但后来我理解了，正是因为寿司已经随处可见，在产品因为工业化率覆盖率极高的情况下，想实现突围，必须要找差异性。



福特给出的方案是用流水线进一步提高产量，拉低单件成本，而日本给出的答案是加入人文属性提高单价。

从工业技术视角来看，前者似乎更有技术，所以大家一直无法理解为什么日式仙人们能长期存活下来。

但从市场经济视角来看，福特流水线和日本的仙人也许并没有高下之分，都是市场环境下优胜劣汰出的方案。黑猫白猫，抓到老鼠就是好猫。



国内现在也一样，在鸡鸭肉禽类产品覆盖率极广的情况下，想实现产品突围，必须要找到新的生产力方案。



贵州养殖户徐其勇就是其中之一，徐其勇原本是一位互联网从业者，有二十年媒体经验，之前在短视频平台梨视频工作，为什么突然跑去养鸡了呢？



因为在一次采访中，他偶然接触到贵州赤水乌骨鸡，然后发现，这个品类的质量极佳，但规模化率却非常低。

这个现象不太合理，但在国内并不罕见，国内有大量优质农产品，还停留在“产业带品牌”的阶段。尤其是供给极易受季节等因素影响的水果，打造品牌更是难事，同是生鲜产品的白鸡，CR10（TOP10品牌的市占率）达45%，但能被人记住品牌名的水果企业，寥寥无几。要知道，以非酒精饮料市场占有率来看，可口可乐也就40%左右，就已经是世界知名的超级品牌了。



而从大众市场来看，这两年大众对大额消费的积极性下降，但对优质生鲜产品的需求却是在快速提升的。

也就是这属于一个，市场天花板在涨，品牌空间也在涨的优质赛道。



而农业，恰好也是这两年供应链大改革，优质生产力大提升的赛道。

直白一点就是，在互联网领域，用AI写文档做PPT已经大规模冒头，卷的没边了，但在传统农业，这些新兴技术还没大量覆盖，如果你恰好是懂前沿技术应用的，那么这里又有一个技术红利。



在综合考量之后，徐其勇决定进入养殖业。



他将目光放在了赤水市的乌骨鸡上，赤水乌骨鸡有300多年养殖历史，含有18种氨基酸，富含人体所需氨基酸、微量元素、矿物质和黑色素，味道鲜美。2016年，农业农村部还将赤水乌骨鸡列入了国家地理标志产品登记保护，产品竞争力本身就极高。

在互联网渠道这块

In [12]:
import openai
openapi_client = openai.Client(base_url="https://qwen72b.gaia.domains/v1",api_key='NA',timeout=1000*60*60)


In [24]:
messages = [
    {"role": "system", "content": SYSTEMP_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]

response = openapi_client.chat.completions.create(
        messages=messages,
        model=MODEL,
        stream=True,
    )
# outputs = response.choices[0].message.content
outputs = ''
for chunk in response:
    print(chunk.choices[0].delta.content,end='')
    outputs += chunk.choices[0].delta.content


# 欢迎收听《仙人与技术》特别节目

**Speaker 1**: 大家好，欢迎收听我们今天的特别节目——《仙人与技术》。我是你们的主持人[您的名字]，今天我们要聊的是一个非常有趣的话题：中国也开始有自己的乌骨鸡“仙人”了。

**Speaker 2**: 哇，乌骨鸡仙人？这听起来太神奇了！我一直觉得仙人这个词用在日本寿司师傅身上已经够奇怪了，没想到现在连养鸡的人都能成为仙人。你是说真的吗？

**Speaker 1**: 当然，这是千真万确的。你知道，以前我看日本的各种寿司仙人、天妇罗仙人时，也是一头雾水。为什么一个随处可见的寿司要弄这么麻烦，什么特制的醋饭，烤过的海苔片？但后来我理解了——正因为寿司已经随处可见，在产品因为工业化覆盖率极高的情况下，想实现突围，必须要找差异性。

**Speaker 2**: 嗯，这倒是真的。福特用流水线进一步提高产量，拉低单件成本，而日本的答案是加入人文属性提高单价。从工业技术视角来看，前者似乎更有技术含量，所以大家一直无法理解为什么日式仙人们能长期存活下来。

**Speaker 1**: 对，但从市场经济的视角来看，福特流水线和日本的仙人并没有高下之分，都是市场环境下优胜劣汰出的方案。黑猫白猫，抓到老鼠就是好猫嘛。

**Speaker 2**: 哈哈，这比喻太棒了！那国内呢？是不是也在做同样的事情？

**Speaker 1**: 是的，国内现在也一样。在鸡鸭肉禽类产品覆盖率极广的情况下，想实现产品突围，必须要找到新的生产力方案。比如贵州养殖户徐其勇就是其中之一。

**Speaker 2**: 哦，这人是谁啊？听起来很有意思！

**Speaker 1**: 徐其勇原本是一位互联网从业者，有二十年媒体经验，之前在短视频平台梨视频工作。为什么突然跑去养鸡了呢？因为在一次采访中，他偶然接触到贵州赤水乌骨鸡，然后发现这个品类的质量极佳，但规模化率却非常低。

**Speaker 2**: 哦，这确实是个机会！那他是怎么做的？

**Speaker 1**: 他将目光放在了赤水市的乌骨鸡上。赤水乌骨鸡有300多年的养殖历史，含有18种氨基酸，富含人体所需氨基酸、微量元素、矿物质和黑色素，味道鲜美。2016年，农业农村部还将赤水乌骨鸡列入了国家地理标志产品登记保护，产品竞争力本身就极高。

**Speaker 2**: 哇，这么

牛逼，我们现在看看效果吧

In [18]:
outputs

'\n\n### 欢迎来到佐治亚州，总统辩论现场直播！\n\n各位听众，欢迎来到我们的播客特别节目！我们今晚将带您进入一场历史性的辩论，这是现任US总统乔·拜登与前US总统唐纳德·特朗普之间的直接对决。两位候选人将在选举日前四个月向美国人民阐述他们的政策和观点。我们非常荣幸地邀请到了主持人杰克·塔珀和达娜·巴什，他们将引导这场激烈的讨论。\n\n### 开场介绍\n\n**Tapper:** 欢迎来到亚特兰大！我是杰克·塔珀，CNN的主播。\n\n**Bush:** 大家好，我是达纳·巴什，CNN政治节目的主持人。今晚我们将共同主持这场辩论。我们的任务是促进两位候选人的对话，并确保观众能够听到他们的真实声音。\n\n### 辩论规则\n\n**Tapper:** 在介绍候选人之前，我们想与在家的观众分享一些辩论规则。前总统特朗普将在屏幕左侧，拜登总统将出现在右侧。每位候选人将有两分钟时间回答问题，每人有一分钟时间进行反驳和澄清。候选人在发言时，麦克风将被打开，对手的麦克风将被关闭，以防止打断。\n\n**Bush:** 辩论结束时，每位候选人将有两分钟的闭幕陈述时间。今晚没有现场观众，辩论期间不允许预先写好的笔记、道具或与竞选工作人员联系。通过接受我们的辩论邀请，候选人和他们的竞选团队都同意这些规则。\n\n### 欢迎两位候选人\n\n**Tapper:** 现在，请欢迎美国第46任总统乔·拜登！\n\n**Tapper:** 请欢迎美国第45任总统唐纳德·特朗普！\n\n**Speaker 3:** 谢谢你们的到来，先生们。让我们开始辩论。\n\n### 经济问题\n\n**Tapper:** 我们从选民最关心的问题——经济开始。拜登总统，通货膨胀已经放缓，但物价仍然很高。自从你上任以来，必需品的价格已经上涨了。例如，一篮子价值一百美元的杂货现在已经超过120美元，典型的房价已经上涨了30%以上。你对那些觉得在你的总统任期内比在特朗普总统任期内更糟糕的选民有什么话要说？\n\n**Biden:** 我们得看看我成为总统后留下了什么。经济处于自由落体状态，疫情处理得非常糟糕，许多人正在死亡。失业率上升至15%，太可怕了。所以我们开始试着把东西重新组合起来。我们创造了15,000个新工作岗位，80万个新的制造业工作岗位，但还有更多工作要做。工人阶级仍然处于困境中。我

让我们将输出结果保存为pickle文件

In [25]:
with open('./resources/data.pkl', 'wb') as file:
    pickle.dump(outputs, file)

In [26]:
with open('./resources/data.txt', 'wb') as file:
    file.write(outputs.encode('utf-8'))

### 下一个 Notebook: 润色讲稿

现在我们已经写好讲稿了，但讲稿还比较粗糙，不够戏剧性和自然。在下一个 Notebook 中，我们将使用 [mlx-community/Qwen2.5-7B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-7B-Instruct-4bit) 模型来实现这一点。

In [21]:
#fin